In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# %%
import torch
import string

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.0 MB/s 


In [4]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 70.1 MB/s 
     |████████████████████████████████| 636 kB 83.0 MB/s 
     |████████████████████████████████| 3.3 MB 58.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##BERT

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [8]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [9]:
text = text["sentence"]

In [10]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [11]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [12]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [13]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [14]:
inputs

{'input_ids': tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [15]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [16]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [17]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [18]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [19]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [20]:
selection[:5]

[[], [6, 7, 15, 16, 17], [], [], [1, 2, 10]]

In [21]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [22]:
inputs.input_ids

tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [23]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [24]:
dataset = MeditationsDataset(inputs)

In [25]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [26]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [27]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [28]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:05<00:00,  1.98s/it, loss=0.0418]


##XLM

In [29]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [30]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [31]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [32]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [33]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [34]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [35]:
mask_arr

tensor([[False, False, False,  ...,  True, False, False],
        [False, False, False,  ...,  True, False,  True],
        [False, False, False,  ...,  True, False,  True],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False, False]])

In [36]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [37]:
selection[:5]

[[7, 14, 16, 17, 18],
 [10, 11, 15, 17, 18, 20],
 [7, 8, 10, 18, 20],
 [13],
 [10, 13, 19]]

In [38]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [39]:
inputs.input_ids

tensor([[    0,    44,    87,  ...,   103,     1,     1],
        [    0,    44, 28811,  ...,   103,     6,   103],
        [    0,    44,  1529,  ...,   103,     1,   103],
        ...,
        [    0,    44, 14838,  ...,     1,     1,     1],
        [    0,    44,  4687,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,   103,     1,     1]])

In [40]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [41]:
dataset = MeditationsDataset(inputs)

In [42]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [43]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [44]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [45]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [03:02<00:00,  5.54s/it, loss=0.249]


In [46]:
bert_model = model.eval()

In [47]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [48]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [49]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [50]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [51]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [52]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [53]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [54]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False,  True],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [55]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [56]:
selection[:5]

[[3, 7], [5, 8, 9, 11, 13, 19], [4, 20], [4, 10, 12, 20], [18]]

In [57]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [58]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,   103,     1],
        [    0,   113,    91,  ...,     1,     1,   103],
        ...,
        [    0,   113, 24841,  ...,   103,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]])

In [59]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [60]:
dataset = MeditationsDataset(inputs)

In [61]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [62]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [63]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [03:52<00:00,  7.06s/it, loss=0.24]


In [64]:
bart_model = bart_model.eval()

##RoBERTa

In [65]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [66]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [67]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [68]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [69]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [70]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [71]:
mask_arr

tensor([[False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False,  True],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False]])

In [72]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [73]:
selection[:5]

[[9, 20], [3, 4, 15, 20], [12, 20], [4, 6, 15, 16], [20]]

In [74]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [75]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,   103],
        [    0,   113,   796,  ..., 39058,     2,   103],
        [    0,   113,    91,  ...,     1,     1,   103],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,   103],
        [    0,     2,     1,  ...,     1,     1,     1]])

In [76]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [77]:
dataset = MeditationsDataset(inputs)

In [78]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [79]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [80]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:12<00:00,  2.21s/it, loss=3.06]


In [81]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [82]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [83]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [84]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [85]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

In [86]:
top_k = 10

In [87]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [88]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [89]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [90]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'tore\nripped\nraced\nswept\nburned',
 'bert': 'ripped\nspread\ntore\nswept\nblazed',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'burned\ntore\nswept\nraced\nspread',
 'xlm': 'runs\npassed\nwent\nrun\nslid',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [91]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'blown\nshattered\nblowing\nfl\nrattled',
 'bert': 'shattered\ncracked\nsmashed\ndark\nblackened',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'shattering\nclosing\ncracking\ncollapsing\ncrumbling',
 'xlm': 'tight\ncold\nhigh\nquiet\nempty',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [92]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'rebuild\nprotect\nrelease\ndraw\nshield',
 'bert': 'save\npreserve\nprotect\nkeep\ntake',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'recover\nemerge\ngrow\nrise\nheal',
 'xlm': 'free\ndefend\nclear\nprotect\nrecover',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [177]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'stimulate\nboost\nfuel\nenerg\nstrengthen',
 'bert': 'boost\ndamage\nstimulate\nhurt\nimpact',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'stimulate\nboost\nignite\ngrow\nhurt',
 'xlm': 'destroy\nhurt\ntax\naffect\nexhaust',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [94]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'bat\nerupt\nblanket\nsw\nsmoke',
 'bert': 'air\nlay\nstood\nsits\nsat',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'condensed\nwrapped\nlies\nenclosed\nflows',
 'xlm': 's\nserved\nappeared\nruns\natmosphere',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia Tech

In [95]:
with open('stockholm/wikipedia_tech/verbs/wiki_tech_verbs_masked.txt', 'r') as fp:
    text = fp.read().split('\n')

In [96]:
text

['\'[\'Technology ("science of craft", from Greek τέχνη, techne, "art, skill, cunning of hand"; and -λογία, -logia) is the sum of techniques, skills, methods, and processes  <mask> in the production of goods or services or in the accomplishment of objectives, such as scientific investigation\',',
 "'Technology  <mask> be the knowledge of techniques, processes, and the like, or it  ',",
 "'<mask> be  <mask> in machines to  <mask> for operation without detailed knowledge of their workings Systems (eg machines)  <mask> technology by  <mask> an input,  <mask> it  ',",
 "'<mask> to the system\\'s use, and then  <mask> an outcome',",
 "'are  <mask> to as technology systems or technological systems\\nThe simplest form of technology is the development and use of basic tools',",
 "'The prehistoric invention of shaped stone tools  <mask> by the discovery of how to  <mask> fire  <',",
 "'mask> sources of food',",
 "'The later Neolithic Revolution  <mask> this, and  <mask> the sustenance available

In [178]:
text_sentence = "['Technology (<mask> of craft, from Greek τέχνη, techne, art, skill, cunning of hand; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives, such as scientific investigation"
get_all_predictions(text_sentence)

['Technology (<mask> of craft, from Greek τέχνη, techne, art, skill, cunning of hand; and -λογία, -logia) is the sum of techniques, skills, methods, and processes used in the production of goods or services or in the accomplishment of objectives, such as scientific investigation


{'bart': 'knowledge\nart\nscience\nmastery\ntechn',
 'bert': 'art\nform\ninvention\nknowledge\ntechnology',
 'electra': 'method\nstyle\ntechnique\nknowledge\ntype',
 'roberta': 'wealth\ncraft\nfull\nhip\nuse',
 'xlm': 'language\nexpression\nፍልስፍና\norganisation\nAssociation',
 'xlnet': '<unk>\nmeans\nword\nlanguage\nkind'}

In [180]:
text_sentence = " <mask> applying technology by taking an input, changing it according to the system\'s use, and then producing an outcome are referred to as technology systems or technological systems"
get_all_predictions(text_sentence)

 <mask> applying technology by taking an input, changing it according to the system's use, and then producing an outcome are referred to as technology systems or technological systems


{'bart': '<s>\nThe\n</s>\nIn\nThese',
 'bert': 'systems\nproducts\ntechnologies\ncompanies\nsolutions',
 'electra': 'systems\nwhen\ndevices\nmethods\napplications',
 'roberta': 'systems\npeople\nMethods\nArticles\nPhones',
 'xlm': 'ສໍານວນ\nПотребители\nThose\nစက်မှု\nSome',
 'xlnet': 'by\nor\nand\nthen\nfor'}

In [181]:
text_sentence = "The prehistoric <mask> of shaped stone tools followed by the discovery of how to control fire increased sources of food"
get_all_predictions(text_sentence)

The prehistoric <mask> of shaped stone tools followed by the discovery of how to control fire increased sources of food


{'bart': 'use\nexcavation\ngrowth\nexploration\nmanufacture',
 'bert': 'discovery\ndevelopment\nintroduction\ninvention\nuse',
 'electra': 'use\ndiscovery\ndevelopment\ninvention\nevolution',
 'roberta': 'development\ninvention\ndiscovery\nuse\ngrowth',
 'xlm': 'study\ndevelopment\nuse\ndesign\nconception',
 'xlnet': 'of\nstyle\nuse\nused\nagain'}

In [182]:
text_sentence = "The invention of the wheel helped humans to travel in and control their <mask>"
get_all_predictions(text_sentence)

The invention of the wheel helped humans to travel in and control their <mask>


{'bart': 'vehicles\ncars\nlocom\nautomobiles\nboats',
 'bert': 'environment\nsurroundings\nenvironments\nlives\ncommunities',
 'electra': 'surroundings\nlives\nactions\nbodies\nenvironment',
 'roberta': 'environment\nworld\nlives\nbodies\ncars',
 'xlm': 'life\nvehicles\nenvironment\nvehicle\ncar',
 'xlnet': 'their\nin\na\nmasters\never'}

In [183]:
text_sentence = "', 'Seagate Technology Holdings plc, an Irish public limited <mask> (commonly referred to as Seagate), is an American data storage company"
get_all_predictions(text_sentence)

', 'Seagate Technology Holdings plc, an Irish public limited <mask> (commonly referred to as Seagate), is an American data storage company


{'bart': 'company\nlimited\npartnership\nliability\ncompanies',
 'bert': 'company\ncorporation\npartnership\nassociation\nliability',
 'electra': 'company\ncorporation\nunit\nfirm\ngroup',
 'roberta': 'company\npartnership\ntrust\nliability\ncorporation',
 'xlm': 'company\npartnership\norganization\ncorporation\ninvestment',
 'xlnet': 'company\ncorporation\nentity\nplc\nsubsidiary'}

In [184]:
text_sentence = "It was incorporated in 1978 as Shugart Technology and commenced <mask> in 1979"
get_all_predictions(text_sentence)

It was incorporated in 1978 as Shugart Technology and commenced <mask> in 1979


{'bart': 'operations\noperation\nproduction\noperating\nconstruction',
 'bert': 'operations\nproduction\noperation\noperating\nbusiness',
 'electra': 'production\ndevelopment\noperating\nmanufacturing\noperations',
 'roberta': 'production\noperations\nbusiness\nsales\noperation',
 'xlm': 'operating\nproduction\nworking\nlife\nresearch',
 'xlnet': 'again\nfurther\noperations\nin\ned'}

In [185]:
text_sentence = " Since 2010, the company has been incorporated in Dublin, Ireland, with operational <mask> in Fremont, California, United States"
get_all_predictions(text_sentence)

 Since 2010, the company has been incorporated in Dublin, Ireland, with operational <mask> in Fremont, California, United States


{'bart': 'offices\nheadquarters\noffice\noperations\nbases',
 'bert': 'offices\nheadquarters\nbase\nfacilities\noffice',
 'electra': 'headquarters\nfacilities\noffices\nlocations\noperations',
 'roberta': 'headquarters\noffices\nbase\nbases\nfacilities',
 'xlm': 'base\ncenter\ncentre\ncentres\noffice',
 'xlnet': 'operations\noffices\nin\ndevelopments\noperating'}

In [186]:
text_sentence = "', 'A technology company (or tech company) is an electronics-based technology company, including, for example, business relating to digital electronics, software, and internet-related <mask>, such as e-commerce services"
get_all_predictions(text_sentence)

', 'A technology company (or tech company) is an electronics-based technology company, including, for example, business relating to digital electronics, software, and internet-related <mask>, such as e-commerce services


{'bart': 'services\nproducts\nactivities\nbusinesses\ncomputer',
 'bert': 'services\nproducts\ntechnologies\nactivities\nbusinesses',
 'electra': 'services\nproducts\napplications\ntechnologies\ntechnology',
 'roberta': 'services\nproducts\nbusinesses\nbusiness\ntechnologies',
 'xlm': 'services\nproducts\nbusiness\nactivities\ntechnologies',
 'xlnet': 'services\nbusinesses\nbusiness\nproducts\nactivities'}

In [187]:
text_sentence = "', 'An <mask> of technology (also referred to as: technological university, technical university, university of technology, technological educational institute, technical college, polytechnic university or just polytechnic) is an institution of tertiary education (such as a university or college) that specializes in engineering, technology, applied science, and natural sciences"
get_all_predictions(text_sentence)

', 'An <mask> of technology (also referred to as: technological university, technical university, university of technology, technological educational institute, technical college, polytechnic university or just polytechnic) is an institution of tertiary education (such as a university or college) that specializes in engineering, technology, applied science, and natural sciences


{'bart': 'institution\ninstitute\nengineering\nacademic\nindependent',
 'bert': 'institute\ninstitution\nuniversity\ninstitutes\nacademy',
 'electra': 'institution\ninstitute\noffice\norganization\ninstitutions',
 'roberta': 'institute\ninstitution\nacademy\nInstitute\nuniversity',
 'xlm': 'institution\nuniversity\ninstitut\nuniversiti\nassociation',
 'xlnet': 'institution\ninstitute\nacademy\nexample\ninstitutions'}

In [188]:
text_sentence = " The term technology comes from the Greek <mask> techne, meaning art and craft, and the word logos, meaning word and speech"
get_all_predictions(text_sentence)

 The term technology comes from the Greek <mask> techne, meaning art and craft, and the word logos, meaning word and speech


{'bart': 'word\nletters\nwords\nterm\nGreek',
 'bert': 'word\nwords\nphrase\nterm\nnoun',
 'electra': 'word\nname\nterm\nwords\ngreek',
 'roberta': 'word\nwords\nroot\nverb\nlanguage',
 'xlm': 'word\nwords\ncapital\nlanguage\nletters',
 'xlnet': 'word\nGreek\nwords\nthe\nlanguage'}

In [189]:
text_sentence = "'Information technology (IT) is the use of computers to create, process, store and exchange all kinds of electronic <mask> and information"
get_all_predictions(text_sentence)

'Information technology (IT) is the use of computers to create, process, store and exchange all kinds of electronic <mask> and information


{'bart': 'data\ninformation\nnews\nfiles\nknowledge',
 'bert': 'information\ndata\nknowledge\nmedia\nnews',
 'electra': 'information\ndata\nmaterials\ntechnology\ndevices',
 'roberta': 'documents\ndata\nfiles\nsignals\ninformation',
 'xlm': 'products\ndocuments\nsystems\nmaterial\ndevices',
 'xlnet': 'and\ndocuments\nmaterials\ndata\nfiles'}

In [190]:
text_sentence = "IT is typically used within the <mask> of business operations as opposed to personal or entertainment technologies"
get_all_predictions(text_sentence)

IT is typically used within the <mask> of business operations as opposed to personal or entertainment technologies


{'bart': 'realm\nframework\ncontext\ncore\ncourse',
 'bert': 'context\nframework\nrealm\ndomain\nfield',
 'electra': 'scope\ncontext\nframework\nfield\nrange',
 'roberta': 'context\nframework\nscope\nconfines\nbounds',
 'xlm': 'field\nareas\nadministration\ncontext\nworld',
 'xlnet': 'of\nthe\ndoors\ncourse\nsystem'}

In [191]:
text_sentence = "IT is considered to be a <mask> of information and communications technology (ICT)"
get_all_predictions(text_sentence)

IT is considered to be a <mask> of information and communications technology (ICT)


{'bart': 'state\nkey\nvital\npowerful\npowerhouse',
 'bert': 'pioneer\nbranch\nprecursor\npart\ncomponent',
 'electra': 'form\nmodel\nbranch\nstudy\nproduct',
 'roberta': 'form\ntype\nprecursor\nbranch\nmanifestation',
 'xlm': 'field\ncomponent\nrevolution\ncombination\npiece',
 'xlnet': 'combination\na\nof\nclass\nthird'}

In [193]:
text_sentence = " An information technology system (IT system) is generally an information system, a communications system, or, more specifically speaking, a computer system – including all <mask>, software, and peripheral equipment – operated by a limited group of IT users"
get_all_predictions(text_sentence)

 An information technology system (IT system) is generally an information system, a communications system, or, more specifically speaking, a computer system – including all <mask>, software, and peripheral equipment – operated by a limited group of IT users


{'bart': 'the\ncomputer\nof\ncomputers\nhard',
 'bert': 'hardware\ncomputer\ninput\ninformation\ncomputers',
 'electra': 'hardware\nsoftware\ncomputers\ndevices\nequipment',
 'roberta': 'hardware\ncomputers\nbooks\nfiles\ncards',
 'xlm': 'component\ninformation\ncomputer\nmaterials\nmaterial',
 'xlnet': 'information\nhardware\nthe\nthat\nequipment'}

In [194]:
text_sentence = "<mask> have been storing, retrieving, manipulating, and communicating information since the Sumerians in Mesopotamia developed writing in about 3000 BC"
get_all_predictions(text_sentence)

<mask> have been storing, retrieving, manipulating, and communicating information since the Sumerians in Mesopotamia developed writing in about 3000 BC


{'bart': '<s>\nThe\n</s>\nAncient\nWe',
 'bert': 'people\nscholars\nwriters\nhumans\nthey',
 'electra': 'they\nwe\ni\narchaeologists\nscientists',
 'roberta': 'people\nPeople\nWe\npeople\nhumans',
 'xlm': 'They\nWe\nThese\nSome\nI',
 'xlnet': 'have\nlong\ninformation\nsince\nthat'}

In [195]:
text_sentence = "We shall call it information <mask> (IT)"
get_all_predictions(text_sentence)

We shall call it information <mask> (IT)


{'bart': 'technology\nTechnology\nmanagement\ntechnologies\nprocessing',
 'bert': '...\nor\nal\nand\ninformation',
 'electra': 'information\nand\nservice\ndata\nfrom',
 'roberta': 'technology\ntechnologies\ntherapy\ntransit\ntech',
 'xlm': 'campaign\ntechnology\nfield\nengine\nofficer',
 'xlnet': 'technology\nsystem\ninformation\nhere\nservice'}

In [196]:
text_sentence = "The term is commonly used as a <mask> for computers and computer networks, but it also encompasses other information distribution technologies such as television and telephones"
get_all_predictions(text_sentence)

The term is commonly used as a <mask> for computers and computer networks, but it also encompasses other information distribution technologies such as television and telephones


{'bart': 'shorthand\ncode\nreference\nterm\ngeneric',
 'bert': 'term\nsynonym\nname\ndesignation\nlabel',
 'electra': 'term\nmetaphor\nsynonym\nreference\nacronym',
 'roberta': 'name\nshorthand\ndescriptor\nterm\nword',
 'xlm': 'term\nsynonym\nname\nprefix\nreference',
 'xlnet': 'term\nmetaphor\nname\ncomputer\nword'}

In [197]:
text_sentence = "Based on the storage and processing technologies employed, it is possible to distinguish four distinct <mask> of IT development: pre-mechanical (3000 BC – 1450 AD), mechanical (1450–1840), electromechanical (1840–1940), and electronic (1940–present)"
get_all_predictions(text_sentence)

Based on the storage and processing technologies employed, it is possible to distinguish four distinct <mask> of IT development: pre-mechanical (3000 BC – 1450 AD), mechanical (1450–1840), electromechanical (1840–1940), and electronic (1940–present)


{'bart': 'stages\nphases\nperiods\neras\nstyles',
 'bert': 'stages\nphases\nperiods\nbranches\neras',
 'electra': 'stages\nmodes\nforms\ntypes\naspects',
 'roberta': 'phases\nstages\neras\nmodes\nprocesses',
 'xlm': 'areas\ntypes\naspects\nsystems\npoints',
 'xlnet': 'stages\ntypes\nphases\nlevels\nperiods'}

In [198]:
text_sentence = "This <mask> focuses on the most recent period (electronic)"
get_all_predictions(text_sentence)

This <mask> focuses on the most recent period (electronic)


{'bart': 'exercise\nstudy\nproject\nexhibit\nanalysis',
 'bert': 'article\nsection\nlist\nbook\npage',
 'electra': 'article\npost\nsection\npaper\nbook',
 'roberta': 'book\npaper\nshow\nanalysis\npiece',
 'xlm': 'study\nexercise\npage\ncampaign\narticle',
 'xlnet': 'focuses\nFocus\nfocus\nI\nfocused'}

In [199]:
text_sentence = " When referred to with its <mask>, EdTech, it is often referring to the industry of companies that create educational technology"
get_all_predictions(text_sentence)

 When referred to with its <mask>, EdTech, it is often referring to the industry of companies that create educational technology


{'bart': 'nickname\nacronym\nname\nbuzz\nterm',
 'bert': 'acronym\nabbreviation\nname\nnickname\ninitials',
 'electra': 'name\npredecessor\nacronym\ntitle\nnamesake',
 'roberta': 'name\nlabel\ntitle\nsuffix\nprefix',
 'xlm': 'name\ncompany\ntechnology\nproduct\norganization',
 'xlnet': 'its\nor\nacronym\ns\na'}

In [200]:
text_sentence = "Massachusetts Institute of Technology (MIT) is a private land-grant research <mask> in Cambridge, Massachusetts"
get_all_predictions(text_sentence)

Massachusetts Institute of Technology (MIT) is a private land-grant research <mask> in Cambridge, Massachusetts


{'bart': 'university\ninstitution\ncenter\ninstitute\nfacility',
 'bert': 'institute\nfacility\nuniversity\ninstitution\ncenter',
 'electra': 'institution\ncenter\ninstitute\nfacility\nagency',
 'roberta': 'university\ninstitute\ninstitution\nschool\ncenter',
 'xlm': 'center\nfacility\ninstitution\ngroup\ncommunity',
 'xlnet': 'agency\ncompany\nproject\nagent\nresearch'}

In [201]:
text_sentence = "The institute has an urban <mask> that extends more than a mile (1"
get_all_predictions(text_sentence)

The institute has an urban <mask> that extends more than a mile (1


{'bart': 'ring\ncampus\ngarden\narea\nstretch',
 'bert': 'footprint\ncore\narea\nlandscape\ncampus',
 'electra': 'area\nradius\npopulation\nstructure\ncommunity',
 'roberta': 'campus\ngarden\nforest\nwall\nplan',
 'xlm': 'environment\ncenter\ncircuit\nbase\ncampus',
 'xlnet': 'that\nbuilding\ndevelopment\ncommunity\nproject'}

In [202]:
text_sentence = "6 km) alongside the Charles River, and encompasses a number of major off-campus <mask> such as the MIT Lincoln Laboratory, the Bates Center, and the Haystack Observatory, as well as affiliated laboratories such as the Broad and Whitehead Institutes"
get_all_predictions(text_sentence)

6 km) alongside the Charles River, and encompasses a number of major off-campus <mask> such as the MIT Lincoln Laboratory, the Bates Center, and the Haystack Observatory, as well as affiliated laboratories such as the Broad and Whitehead Institutes


{'bart': 'laboratories\nresearch\nlabs\nobserv\nfacilities',
 'bert': 'facilities\nbuildings\nlaboratories\ninstitutions\nvenues',
 'electra': 'laboratories\nfacilities\nlabs\nprojects\ninstitutions',
 'roberta': 'institutions\nfacilities\nlabs\ncenters\nlaboratories',
 'xlm': 'facilities\nprojects\nsites\ncentres\nareas',
 'xlnet': 'laboratories\nfacilities\ninstitutions\nbuildings\nlabs'}

In [203]:
text_sentence = " As of June 2021, 97 Nobel <mask>, 26 Turing Award winners, and 8 Fields Medalists have been affiliated with MIT as alumni, faculty members, or researchers"
get_all_predictions(text_sentence)

 As of June 2021, 97 Nobel <mask>, 26 Turing Award winners, and 8 Fields Medalists have been affiliated with MIT as alumni, faculty members, or researchers


{'bart': 'laure\nWinners\nLaure\nlaureate\nPrize',
 'bert': 'laureate\nwinners\nprizes\ns\nscholars',
 'electra': 'winners\nprizes\ns\nlaureate\nfellows',
 'roberta': 'prizes\nwinners\nPrize\nmentions\nawards',
 'xlm': 'colleagues\nstudents\nteachers\nemployees\nparties',
 'xlnet': 'winner\ns\nwinners\na\nas'}

In [204]:
text_sentence = " In addition, 58 National Medal of Science <mask>, 29 National Medals of Technology and Innovation recipients, 50 MacArthur Fellows, 80 Marshall Scholars, 3 Mitchell Scholars, 22 Schwarzman Scholars, 9 Knight-Hennessy Scholars,  41 astronauts, and 16 Chief Scientists of the U, Air Force have been affiliated with MIT"
get_all_predictions(text_sentence)

 In addition, 58 National Medal of Science <mask>, 29 National Medals of Technology and Innovation recipients, 50 MacArthur Fellows, 80 Marshall Scholars, 3 Mitchell Scholars, 22 Schwarzman Scholars, 9 Knight-Hennessy Scholars,  41 astronauts, and 16 Chief Scientists of the U, Air Force have been affiliated with MIT


{'bart': 'winners\nWinners\nrecipients\nfinalists\nand',
 'bert': 'recipients\nwinners\nrecipient\nlaureate\nawards',
 'electra': 'recipients\nrecipient\nwinners\ndonors\nholders',
 'roberta': 'recipients\nawards\nwinners\nAwards\nnominees',
 'xlm': 'students\nemployees\nteachers\ns\nrecipient',
 'xlnet': 'recipients\nwinners\nawards\nresearchers\nrecipient'}

In [205]:
text_sentence = "MIT is a <mask> of the Association of American Universities (AAU)"
get_all_predictions(text_sentence)

MIT is a <mask> of the Association of American Universities (AAU)


{'bart': 'member\nfounding\ncharter\nMember\nbranch',
 'bert': 'member\ncomponent\npart\nchapter\nconstituent',
 'electra': 'member\nbranch\ndivision\nsubsidiary\nunit',
 'roberta': 'member\nconstituent\nMember\nmembership\nbranch',
 'xlm': 'member\ncomponent\npart\ndivision\ngroup',
 'xlnet': 'member\nstudent\npart\na\none'}

In [206]:
text_sentence = "Gigabyte <mask> (branded as GIGABYTE or sometimes GIGA-BYTE; formally GIGA-BYTE Technology Co"
get_all_predictions(text_sentence)

Gigabyte <mask> (branded as GIGABYTE or sometimes GIGA-BYTE; formally GIGA-BYTE Technology Co


{'bart': 'Boost\nTechnologies\nLaboratories\nLLC\nTechnology',
 'bert': 'technology\ntechnologies\ncomputing\ntechnological\nict',
 'electra': 'technology\ntechnologies\nco\ncompany\ncorporation',
 'roberta': 'XY\nSpectrum\nfused\nToys\nSlim',
 'xlm': 'technology\nTechnology\ns\ntechnologies\ninformation',
 'xlnet': '”\ns\nand'}

In [207]:
text_sentence = ") is a Taiwanese <mask> and distributor of computer hardware"
get_all_predictions(text_sentence)

) is a Taiwanese <mask> and distributor of computer hardware


{'bart': 'manufacturer\nseller\nmaker\ncompany\ndeveloper',
 'bert': 'manufacturer\nproducer\nretailer\ncompany\nmaker',
 'electra': 'manufacturer\nproducer\ndeveloper\nauthor\nowner',
 'roberta': 'designer\nmanufacturer\nproducer\ninventor\ndealer',
 'xlm': 'owner\ndesigner\nproducer\nconsumer\nauthor',
 'xlnet': 'and\nmanufacturer\ncompany\ndistributor\nproducer'}

In [208]:
text_sentence = "8 million <mask> in the first quarter of 2015, which allowed it to become the leading motherboard vendor"
get_all_predictions(text_sentence)

8 million <mask> in the first quarter of 2015, which allowed it to become the leading motherboard vendor


{'bart': 'units\ndollars\nPCs\nCPUs\ncomputers',
 'bert': 'downloaded\nsold\nusers\nregistered\nshipped',
 'electra': 'dollars\ncustomers\nsold\npeople\nrose',
 'roberta': 'sales\norders\nunits\ndollars\ndownloads',
 'xlm': 'visitors\ncustomers\nusers\nχρήστες\npeople',
 'xlnet': 'in\ndollars\nmore\npeople\nusers'}

In [209]:
text_sentence = " Gigabyte also manufactures custom graphics <mask> and laptop computers (including thin and light laptops under its Aero sub-brand)"
get_all_predictions(text_sentence)

 Gigabyte also manufactures custom graphics <mask> and laptop computers (including thin and light laptops under its Aero sub-brand)


{'bart': 'cards\nchips\nprocessors\nadapters\nfor',
 'bert': 'desktop\ntablets\ntablet\nboards\nconsoles',
 'electra': 'machines\ncards\ncomputers\ndevices\nsystems',
 'roberta': 'cards\nprocessors\nchips\nadapters\nboards',
 'xlm': 'novel\nbooks\ndesign\nproducts\nsystems',
 'xlnet': 'and\ngraphics\nchips\nchip\noutput'}

In [210]:
text_sentence = " In 2010, Gigabyte was ranked 17th in Taiwan's Top 20 Global <mask> by the Taiwan External Trade Development Council"
get_all_predictions(text_sentence)

 In 2010, Gigabyte was ranked 17th in Taiwan's Top 20 Global <mask> by the Taiwan External Trade Development Council


{'bart': 'Companies\nbrands\nLeaders\ncompanies\nBrands',
 'bert': 'companies\nbrands\nbusinesses\nproducts\nemployers',
 'electra': 'markets\neconomies\nindices\ncountries\nnations',
 'roberta': 'Brands\ncompanies\nMarkets\nPlayers\nCapitals',
 'xlm': 'marketplace\nMarket\nSite\nPortal\nNews',
 'xlnet': 'Global\nTen\ns\nDevices\nList'}

In [211]:
text_sentence = "The <mask> is publicly held and traded on the Taiwan Stock Exchange, stock ID number TWSE: 2376"
get_all_predictions(text_sentence)

The <mask> is publicly held and traded on the Taiwan Stock Exchange, stock ID number TWSE: 2376


{'bart': 'company\nstock\nfirm\nshares\nholding',
 'bert': 'company\nstock\nbank\nfirm\nshares',
 'electra': 'stock\ncompany\nproduct\nshares\ntitle',
 'roberta': 'company\nstock\nfirm\nbusiness\nplant',
 'xlm': 'investment\nsupplement\nbroker\ndividend\ncapital',
 'xlnet': 'is\nI\nIt\nThe\nExchange'}

In [213]:
text_sentence = "The Indian <mask> of Technology (IITs) are autonomous public technical universities located across India"
get_all_predictions(text_sentence)

The Indian <mask> of Technology (IITs) are autonomous public technical universities located across India


{'bart': 'Institutes\nInstitute\nIndian\nInstit\nColleges',
 'bert': 'institutes\ninstitute\nuniversities\ninstitut\ncolleges',
 'electra': 'institute\nacademy\ninstitutes\nuniversity\ndepartment',
 'roberta': 'Institutes\nInstitute\nSchools\nUniversities\nschools',
 'xlm': 'Institute\nInstitution\nUniversity\nUniversiti\nInstitut',
 'xlnet': 'Institute\nUniversity\nState\nTrust\nAct'}

In [214]:
text_sentence = " The <mask> of Technology Act, 1961 lists twenty-three institutes"
get_all_predictions(text_sentence)

 The <mask> of Technology Act, 1961 lists twenty-three institutes


{'bart': 'Indian\nGraduate\nFaculty\nInstitute\nCollege',
 'bert': 'institute\ndepartment\ninstitutes\nadvancement\nuse',
 'electra': 'institute\nuniversity\nstate\nestablishment\nadvancement',
 'roberta': 'Nation\nScience\nInstitute\nTransfer\nInstitutes',
 'xlm': 'House\nSecretary\nInstitute\nDepartment\nReview',
 'xlnet': 's\nof\nOf\nNumber\nAct'}

In [215]:
text_sentence = "The <mask> of Education is the ex officio Chairperson of the IIT Council"
get_all_predictions(text_sentence)

The <mask> of Education is the ex officio Chairperson of the IIT Council


{'bart': 'Minister\nMaharashtra\nstate\nSecretary\nUnion',
 'bert': 'minister\nsecretary\ndirector\ncommissioner\nministry',
 'electra': 'minister\nsecretary\ndirector\ncommissioner\nsuperintendent',
 'roberta': 'Minister\nSuperintendent\nDean\nminister\nSecretary',
 'xlm': 'Secretary\nMinister\nminister\nHead\nMinistry',
 'xlnet': 'The\nIt\nCourse\nthe\nSchool'}

In [217]:
text_sentence = "As of 2020, the total number of <mask> for undergraduate programs in all IITs is 16,053"
get_all_predictions(text_sentence)

As of 2020, the total number of <mask> for undergraduate programs in all IITs is 16,053


{'bart': 'students\nseats\nscholarships\nadmissions\nspots',
 'bert': 'students\napplications\napplicants\ncourses\nprofessors',
 'electra': 'applicants\nstudents\ngraduates\napplications\nenrollment',
 'roberta': 'students\napplications\napplicants\nadmissions\nseats',
 'xlm': 'students\napplications\nschools\nplaces\nopportunities',
 'xlnet': 'applicants\npeople\nfunding\nplaces\navailable'}

In [218]:
text_sentence = " is an American producer of computer memory and computer data storage including dynamic random-access <mask>, flash memory, and USB flash drives"
get_all_predictions(text_sentence)

 is an American producer of computer memory and computer data storage including dynamic random-access <mask>, flash memory, and USB flash drives


{'bart': 'ors\ncomputer\nand\nspots\nfiles',
 'bert': 'memory\nstorage\ntape\ndrives\nram',
 'electra': 'memory\ndevices\ndrives\nstorage\nmedia',
 'roberta': 'cards\nmemory\nkeys\ndrives\ndisks',
 'xlm': 'memory\ndrive\nsolutions\nsystems\nprograms',
 'xlnet': 'memory\ntype\nmemories\nsuch\nmedia'}

In [219]:
text_sentence = " It is headquartered in <mask>, Idaho"
get_all_predictions(text_sentence)

 It is headquartered in <mask>, Idaho


{'bart': 'Boise\nIdaho\nBeaver\nMeridian\nSpokane',
 'bert': 'boise\nmoscow\nlima\njerome\nidaho',
 'electra': 'boise\nidaho\nspokane\nportland\nogden',
 'roberta': 'Boise\nAda\nMeridian\nMoscow\nCaldwell',
 'xlm': 'Shinyanga\nMagharibi\nParíž\nVijayawada\nPonteareas',
 'xlnet': 'in\nthe\na\nIn\nin'}

In [220]:
text_sentence = "Its consumer <mask> are marketed under the brands Crucial  and Ballistix"
get_all_predictions(text_sentence)

Its consumer <mask> are marketed under the brands Crucial  and Ballistix


{'bart': 'products\nappliances\nelectronics\ngoods\nparts',
 'bert': 'products\ngoods\nelectronics\nbrands\nkits',
 'electra': 'products\nbrands\ngoods\nservices\nitems',
 'roberta': 'products\ngoods\nitems\nelectronics\ncards',
 'xlm': 's\nproducts\ngroups\nprices\nbase',
 'xlnet': 'are\nproducts\nis\nwho\n’'}

In [221]:
text_sentence = " Micron and Intel together created IM Flash Technologies, which produces NAND flash <mask>"
get_all_predictions(text_sentence)

 Micron and Intel together created IM Flash Technologies, which produces NAND flash <mask>


{'bart': 'chips\n."\ncrystals\nmemory\ndrives',
 'bert': 'products\nmemory\nflash\ntechnology\nmemories',
 'electra': 'technologies\ntechnology\napplications\ndevices\ncontent',
 'roberta': 'cards\nproducts\nchips\ndrives\nsolutions',
 'xlm': 'light\nmemory\nflood\nsolutions\ndrive',
 'xlnet': 'flash\nblocking\nand\ncurrent\nconverting'}

In [222]:
text_sentence = "It owned <mask> between 2006 and 2017"
get_all_predictions(text_sentence)

It owned <mask> between 2006 and 2017


{'bart': 'around\nproperty\nseveral\nthe\nmillions',
 'bert': 'shares\nland\nproperty\ncompanies\nstock',
 'electra': 'it\nbusiness\nall\nbusinesses\nland',
 'roberta': 'land\neverything\nproperty\nRover\nher',
 'xlm': 'wrinkles\npoverty\nvehicles\nସେନା\npassion',
 'xlnet': 'in\nwas\nsaid\non\nheld'}